# Introduction

1. Training on cartplot, should be easy
2. Entropy on middle layer

In [1]:
import gym, time
import numpy as np
import matplotlib.pyplot as plt
from agent import Agent
from collections import Counter
%matplotlib inline

#Environment
seed = 0
env = gym.make('CartPole-v0')
env.seed(seed)  # for comparison
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n

#Agent
gamma, lr, tau = 0.99, 0.001, 0.0001
agent = Agent(num_states, num_actions, lr, gamma, seed_num = seed)
agent.memory_size = 10**4
agent.batchsize = 32
learning_start = 2000
agent.tau = tau


#Train
EPISODES = 300
scores = []
t1 = time.time()
for e in range(1,EPISODES+1):
    state = env.reset()
    reward_sum = 0
    done = False
    steps = 0
    actions = []
    
    while not done:
        #env.render()
        state = np.reshape(state, [1, num_states])  #reshape for keras
        action_onehot = agent.act(state)
        action_scalar = np.dot(action_onehot,range(num_actions))
        actions.append(action_scalar)
        next_state, reward, done, _ = env.step(action_scalar)
        reward_sum += reward
        agent.remember(state[0], action_onehot, reward, next_state, done)
        state = next_state
        
        if len(agent.memory) > learning_start:
            agent.train_models()
            agent.actor.gumbel_temperature = max(0.999*agent.actor.gumbel_temperature, 0.1)
        steps += 1
    
    #Learn & print results
    scores.append(reward_sum)
    if e % 10 == 0:
        t2 = time.time()
        print '(episode, score, T (mins)) = ' + str((e,reward_sum, (t2-t1)/60.0))
        print 'actions counts = ' + str(Counter(actions)) + '\n'


#agent.save_target_weights()
plt.plot(scores)
#np.savetxt('stats/scores_inverted_pendulum.txt',scores)

/home/kokeeffe/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
(episode, score, T (mins)) = (10, 11.0, 0.001473716894785563)
actions counts = Counter({1: 8, 0: 3})

(episode, score, T (mins)) = (20, 19.0, 0.0028935988744099935)
actions counts = Counter({1: 14, 0: 5})

(episode, score, T (mins)) = (30, 22.0, 0.004326148827870687)
actions counts = Counter({1: 13, 0: 9})

(episode, score, T (mins)) = (40, 22.0, 0.005235882600148519)
actions counts = Counter({0: 12, 1: 10})

(episode, score, T (mins)) = (50, 12.0, 0.0063334306081136065)
actions counts = Counter({0: 7, 1: 5})

(episode, score, T (mins)) = (60, 11.0, 0.007765515645345052)
actions counts = Counter({1: 7, 0: 4})

(episode, score, T (mins)) = (70, 14.0, 0.00943899949391683)
actions counts = Counter({1: 11, 0: 3})

(episode, score, T (mins)) = (80, 12.0, 0.011237080891927083)
actions counts = Counter({1: 9, 0: 3})

(episode, score, T (mins)) = (90, 20.0, 0.012454366683959961)
actions counts = C

InvalidArgumentError: Incompatible shapes: [32] vs. [32,2]
	 [[Node: gradients/sub_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](gradients/sub_grad/Shape, gradients/sub_grad/Shape_1)]]

### Watch a smart agent

In [5]:
env.reset()
done = False
while not done:
    env.render()
    state = np.reshape(state, [1, num_states])  #reshape for keras
    action_onehot = agent.act(state)
    action_scalar = np.dot(action_onehot,range(num_actions))
    next_state, reward, done, _ = env.step(action_scalar)
    reward_sum += reward
    state = next_state
    #print 'position = ' + str( state[0] )
env.close()